In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import tensorflow as tf
from keras.models import Sequential
import numpy as np
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
import np_utils
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [29]:
df_test = pd.read_csv('/kaggle/input/data-2/test.csv')
df=pd.read_csv('/kaggle/input/data-2/train.csv')

In [ ]:
df

In [30]:
#Create a column with all the input information and removing any info that consists NaN
df['total'] = (df['title'].fillna('') + ' ' + df['abstract'].fillna('') + ' ' + df['body'].fillna('')).str.strip()
df_test['total'] = (df_test['title'].fillna('') + ' ' + df_test['abstract'].fillna('') + ' ' + df_test['body'].fillna('')).str.strip()

In [34]:
#turn the text values of output column to numerical values
mapping = {
    'آشپزی': 0,
    'اجتماعی': 1,
    'اخبار سلامت': 2,
    'اقتصادی': 3,
    'بین الملل': 4,
    'حوادث': 5,
    'خواندنی ها و دیدنی ها': 6,
    'داستان کوتاه': 7,
    'دانلود': 8,
    'روانشناسی': 9,
    'سرگرمی': 10,
    'سفر و تفریح': 11,
    'سلامت': 12,
    'سیاست خارجی': 13,
    'سیاسی': 14,
    'علمی': 15,
    'عمومی': 16,
    'فرهنگی/هنری': 17,
    'فناوری و IT': 18,
    'محیط زیست': 19,
    'ورزشی': 20,
    'پیامک': 21,
    'کاربران': 22
}

df['val_subgroup'] = df['subgroup'].map(mapping)
import re

In [35]:
def clean_total_column(df):
    """
    Cleans the 'total' column of a DataFrame by removing '\r\n' characters, English, and Persian numbers.

    Parameters:
        df (pandas.DataFrame): DataFrame containing the 'total' column.

    Returns:
        pandas.DataFrame: DataFrame with the 'total' column cleaned.
    """
    # Remove '\r\n' characters
    df['total'] = df['total'].str.replace('\r\n', '')

    # Define a regular expression to match both English and Persian numbers
    number_pattern = re.compile(r'[0-9۰-۹]+')

    # Remove numbers from the 'total' column
    df['total'] = df['total'].apply(lambda x: re.sub(number_pattern, '', x))

    return df

In [36]:
clean_total_column(df)

,title,subgroup,abstract,body,total,val_subgroup
0,پلیس: جرائم خشن و مسلحانه در تهران کاهش یافته است,عمومی,آمارهای پلیس نشان می‌دهد که جرائم خشن و مسلحان...,رئیس پلیس آگاهی تهران بزرگ اعلام کرد که موضوع ...,پلیس: جرائم خشن و مسلحانه در تهران کاهش یافته ...,16
1,"وزیر بهداشت:\r\nآغاز اجرای طرح جامع ""دارویار""/...",اخبار سلامت,هدف اصلی وزارت بهداشت از این طرح این است که پو...,"وزیر بهداشت جزییات طرح ""دارویار"" که اجرای آن آ...","وزیر بهداشت:آغاز اجرای طرح جامع ""دارویار""/ افز...",2
2,وزارت بهداشت: قیمت دارو برای مصرف کننده ثابت م...,اجتماعی,NaN,سخنگوی وزارت بهداشت، درمان و آموزش پزشکی در تو...,وزارت بهداشت: قیمت دارو برای مصرف کننده ثابت م...,1
3,معاون رئیسی: مردم به زودی شاهد اثرات مثبت اقدا...,سیاسی,معاون امور مجلس رئیس جمهور: استان گیلان دارای ...,معاون رئیس جمهور گفت: دولت برنامه های راهبردی ...,معاون رئیسی: مردم به زودی شاهد اثرات مثبت اقدا...,14
4,دستگیری سارق ۲۰ هزار دلاری ارز دیجیتال در گلستان,اجتماعی,شهروندان توصیه‌های پلیس فتا را جدی بگیرند و در...,رییس پلیس فتا فرماندهی انتظامی گلستان گفت: سار...,دستگیری سارق هزار دلاری ارز دیجیتال در گلستان...,1
...,...,...,...,...,...,...
237879,معاون وزیر ارشاد: مخالف انحصار صداوسیما هستیم,سیاسی,قطعا با هرگونه انحصار در فعالیت‌های فراگیر مرد...,معاون مطبوعاتی وزیر ارشاد گفت: انحصار صدور مجو...,معاون وزیر ارشاد: مخالف انحصار صداوسیما هستیم ...,14
237880,دیدار دست‌اندرکاران همایش حکیم تهران با مقام م...,سیاسی,حذف فلسفه از حوزه علمیه قم موجب خواهد شد افراد...,پایگاه اطلاع رسانی مقام معظم رهبری نوشت: بیانا...,دیدار دست‌اندرکاران همایش حکیم تهران با مقام م...,14
237881,ترامپ: هنگام امضای برجام باید به سوریه و یمن ن...,سیاست خارجی,ترامپ برجام را یک توافق بد توصیف و تصریح کرد: ...,رئیس‌جمهور آمریکا بار دیگر با انتقاد از برجام ...,ترامپ: هنگام امضای برجام باید به سوریه و یمن ن...,13
237882,واردات گوشی تلفن همراه از خودرو جلو زد,اقتصادی,در فروردین ماه امسال­­ تعداد 249 هزار و 154 دس...,براساس آمار گمرک ایران در­ فروردین ماه امسال ­...,واردات گوشی تلفن همراه از خودرو جلو زد در فرور...,3


In [37]:
labels = df['val_subgroup'].values


In [38]:
labels

array([16,  2,  1, ..., 13,  3,  3])

In [39]:
#Tokenization of input data
X = df['total']
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
X=tokenizer.texts_to_sequences(X)

In [40]:
X=pad_sequences(X,maxlen=100)

In [41]:
y_train_cat = to_categorical(y_train, 23)
y_test_cat = to_categorical(y_test, 23)

In [42]:
X.shape

(237884, 100)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=1)

In [44]:
from tensorflow.keras import layers

In [45]:
import tensorflow as tf
from tensorflow.keras import layers

# Confirm GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define your model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=20000, output_dim=32, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Bidirectional(layers.LSTM(256)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(23, activation='softmax')
])

# Compile your model
rnn_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adamax(),
                  metrics=['accuracy'])

# Train your model
rnn_model.fit(X_train, y_train_cat, epochs=10,
              validation_data=(X_test, y_test_cat),
              validation_steps=30,
              verbose=1)


Num GPUs Available:  2
Epoch 1/10
5948/5948 [==============================] - 199s 32ms/step - loss: 1.4343 - accuracy: 0.5421 - val_loss: 1.2052 - val_accuracy: 0.6257
Epoch 2/10
5948/5948 [==============================] - 170s 29ms/step - loss: 1.0544 - accuracy: 0.6649 - val_loss: 0.9904 - val_accuracy: 0.6789
Epoch 3/10
5948/5948 [==============================] - 170s 28ms/step - loss: 0.9329 - accuracy: 0.6980 - val_loss: 0.9340 - val_accuracy: 0.6974
Epoch 4/10
5948/5948 [==============================] - 169s 28ms/step - loss: 0.8740 - accuracy: 0.7123 - val_loss: 0.9079 - val_accuracy: 0.7011
Epoch 5/10
5948/5948 [==============================] - 169s 28ms/step - loss: 0.8303 - accuracy: 0.7232 - val_loss: 0.8897 - val_accuracy: 0.7080
Epoch 6/10
5948/5948 [==============================] - 169s 28ms/step - loss: 0.7963 - accuracy: 0.7322 - val_loss: 0.8767 - val_accuracy: 0.7096
Epoch 7/10
5948/5948 [==============================] - 169s 28ms/step - loss: 0.7683 - accurac

In [46]:
# Save the weights of the RNN model
rnn_model.save_weights('rnn_model_weights.h5')


In [52]:
df_test['total'] = (df_test['title'].fillna('') + ' ' + df_test['abstract'].fillna('') + ' ' + df_test['body'].fillna('')).str.strip()

In [53]:
df_test

,title,abstract,body,total
0,رئیس قوه قضائیه: دشمنان در مقام این هستند که ق...,امروز ملاحظه می‌کنید که تمام دشمنان و سلطه جها...,رییس قوه قضاییه گفت: امروز ملاحظه می کنید که ت...,رئیس قوه قضائیه: دشمنان در مقام این هستند که ق...
1,برخورد شهرداری شیراز با مالکی که درخت توت کهنس...,NaN,NaN,برخورد شهرداری شیراز با مالکی که درخت توت کهنس...
2,آیت‌الله جنتی: باید از مأموران نیروی انتظامی د...,باید از مأموران نیروی انتظامی در زمینه برخورد ...,دبیر شورای نگهبان با اظهار نگرانی درباره مسئله...,آیت‌الله جنتی: باید از مأموران نیروی انتظامی د...
3,امدادرسانی به عشایر گرفتار در سیل -چهارمحال و ...,با توجه به بارش باران در استان چهارمحال و بختی...,منبع: تسنیم,امدادرسانی به عشایر گرفتار در سیل -چهارمحال و ...
4,شمخانی: گفت‌وگوی مجدد درباره فعالیت‌های هسته‌ا...,متأسفانه امروز ما با دولتی مواجه هستیم که نه ت...,دبیر شورای عالی امنیت ملی تاکید کرد: هر توافقی...,شمخانی: گفت‌وگوی مجدد درباره فعالیت‌های هسته‌ا...
...,...,...,...,...
101945,پنتاگون: به نیروهای بسیج مردمی عراق در آزادساز...,نیروهای عراقی و هم پیمانان آنها طی دو هفته گذش...,وزرات دفاع آمریکا (پنتاگون) اعلام کرد:‌ آمریکا...,پنتاگون: به نیروهای بسیج مردمی عراق در آزادساز...
101946,از فوتبال و والیبال تا خودروسازی؛ آیا نیازمند ...,در برابر این عده باید گفت چگونه است که در امور...,بعد از برجام، خودروسازهای خارجی به سراغ همتایا...,از فوتبال و والیبال تا خودروسازی؛ آیا نیازمند ...
101947,چین به کپی‌برداری موشکی متهم شد,طرح جنگنده 1.44 فرصت جهش بزرگ در صنعت هواپیمای...,NaN,چین به کپی‌برداری موشکی متهم شد طرح جنگنده 1.4...
101948,چای سبز چقدر و چه زمانی باید مصرف شود,متخصصان تغذیه ، نوشیدن چای سبز با غذا را توصیه...,شکی نیست که چای سبز سالم‌ترین نوشیدنی است. این...,چای سبز چقدر و چه زمانی باید مصرف شود متخصصان ...


In [55]:
clean_total_column(df_test)

,title,abstract,body,total
0,رئیس قوه قضائیه: دشمنان در مقام این هستند که ق...,امروز ملاحظه می‌کنید که تمام دشمنان و سلطه جها...,رییس قوه قضاییه گفت: امروز ملاحظه می کنید که ت...,رئیس قوه قضائیه: دشمنان در مقام این هستند که ق...
1,برخورد شهرداری شیراز با مالکی که درخت توت کهنس...,NaN,NaN,برخورد شهرداری شیراز با مالکی که درخت توت کهنس...
2,آیت‌الله جنتی: باید از مأموران نیروی انتظامی د...,باید از مأموران نیروی انتظامی در زمینه برخورد ...,دبیر شورای نگهبان با اظهار نگرانی درباره مسئله...,آیت‌الله جنتی: باید از مأموران نیروی انتظامی د...
3,امدادرسانی به عشایر گرفتار در سیل -چهارمحال و ...,با توجه به بارش باران در استان چهارمحال و بختی...,منبع: تسنیم,امدادرسانی به عشایر گرفتار در سیل -چهارمحال و ...
4,شمخانی: گفت‌وگوی مجدد درباره فعالیت‌های هسته‌ا...,متأسفانه امروز ما با دولتی مواجه هستیم که نه ت...,دبیر شورای عالی امنیت ملی تاکید کرد: هر توافقی...,شمخانی: گفت‌وگوی مجدد درباره فعالیت‌های هسته‌ا...
...,...,...,...,...
101945,پنتاگون: به نیروهای بسیج مردمی عراق در آزادساز...,نیروهای عراقی و هم پیمانان آنها طی دو هفته گذش...,وزرات دفاع آمریکا (پنتاگون) اعلام کرد:‌ آمریکا...,پنتاگون: به نیروهای بسیج مردمی عراق در آزادساز...
101946,از فوتبال و والیبال تا خودروسازی؛ آیا نیازمند ...,در برابر این عده باید گفت چگونه است که در امور...,بعد از برجام، خودروسازهای خارجی به سراغ همتایا...,از فوتبال و والیبال تا خودروسازی؛ آیا نیازمند ...
101947,چین به کپی‌برداری موشکی متهم شد,طرح جنگنده 1.44 فرصت جهش بزرگ در صنعت هواپیمای...,NaN,چین به کپی‌برداری موشکی متهم شد طرح جنگنده . ف...
101948,چای سبز چقدر و چه زمانی باید مصرف شود,متخصصان تغذیه ، نوشیدن چای سبز با غذا را توصیه...,شکی نیست که چای سبز سالم‌ترین نوشیدنی است. این...,چای سبز چقدر و چه زمانی باید مصرف شود متخصصان ...


In [56]:
X = df_test['total']
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
X=tokenizer.texts_to_sequences(X)
X_test_final=pad_sequences(X,maxlen=100)

In [59]:
predictions = rnn_model.predict(X_test_final)

3186/3186 [==============================] - 31s 10ms/step


In [60]:
predicted_indices = predictions.argmax(axis=1)
predicted_subgroups = [list(mapping.keys())[list(mapping.values()).index(idx)] for idx in predicted_indices]
df_test['val_subgroup'] = predicted_subgroups
df_test['subgroup'] = df_test['val_subgroup'].map(mapping)


In [62]:
df_test